若被ban IP可以參考此

In [1]:
import requests
from bs4 import BeautifulSoup
from sys import stdout
import numpy as np
import re
import pandas as pd
import time
import math
from fake_useragent import UserAgent
from stem import Signal
from stem.control import Controller

import twstock

In [2]:
# get twse stock codes
code_dict = twstock.twse

code_list = []
for c in code_dict.keys():
    if c[0] != '0' and c.isdigit():
        code_list.append(c)

In [6]:

# crawler settings
ua = UserAgent()
headers = {'User-Agent': ua.random}
proxies = {
    'http': 'socks5://127.0.0.1:9150',
    'https': 'socks5://127.0.0.1:9150'
} # tor的port，要同時開起來tor browser，要開啟start tor server and set port
# most retrying time
try_time = 5


# 拿到ban訊息，就執行這個fn去換IP
def get_new_ip():
    headers = {'User-Agent': ua.random}
    with Controller.from_port(port=9051) as controller:
        controller.authenticate(password="oiejrgeoibeifj") #tor pwd?
        controller.signal(Signal.NEWNYM)
        time.sleep(controller.get_newnym_wait())
    r = requests.get('http://orange.tw/', proxies=proxies) #當前IP
    return r.text.replace('\n', " ")

In [4]:
# parameters setting
## 幾個月內的歷史資料
history_num_of_month = 12

In [7]:
history_data_df = pd.DataFrame(columns=['stock_id', 'lowest', 'total_money_amount'])
count = 0
current_ip = get_new_ip()

for index, stock_id in enumerate(code_list):
    stdout.write('\rgetting history data of %s (%d/%d) with %s' % (stock_id, index, len(code_list)-1, current_ip))
    stdout.flush()
    for i in range(try_time):
        try:
            r = requests.get('https://goodinfo.tw/StockInfo/ShowK_Chart.asp?STOCK_ID=' + stock_id + '&CHT_CAT2=MONTH', 
                             headers=headers, proxies=proxies) #
            soup = BeautifulSoup(r.content, 'lxml')

            monthly_lowest = []
            monthly_total_money_amount = 0
            for i in range(history_num_of_month):
                row = soup.find_all('tr', {'id':'row'+str(i)})
                column = row[0].find_all('td')
                monthly_lowest.append(float(column[4].text))
                monthly_total_money_amount = monthly_total_money_amount + float(column[10].text)
            history_data_df = history_data_df.append({'stock_id': stock_id, 'lowest': min(monthly_lowest), 
                                                      'total_money_amount': monthly_total_money_amount},
                                                     ignore_index=True)
            break
        except:
            soup = BeautifulSoup(r.content, 'lxml')
            if '您的瀏覽量異常' in soup.text:
                print(' <----- ERROR (' + soup.text.replace('\n', '')[:50] + '), getting new ip.')
                current_ip = get_new_ip()
            else:
                print(' <----- ERROR (' + soup.text.replace('\n', '')[:100] + ')')
                break

getting history data of 1471 (94/916) with 176.10.99.200 Switzerland  <----- ERROR (您的瀏覽量異常, 已影響網站速度, 目前暫時關閉服務, 請稍後再重新使用若您是使用程式大量下載本網站), getting new ip.
getting history data of 2317 (265/916) with 163.172.223.200 United Kingdom  <----- ERROR (  (2317) 鴻海 技術分析 - 月K線圖暨股價漲跌資料表 - Goodinfo!台灣股市資訊網  Goodinfo!台灣股市資訊網股票代號/名稱    選擇登入方式   googletag.cm)
getting history data of 2330 (273/916) with 163.172.223.200 United Kingdom  <----- ERROR (  (2330) 台積電 技術分析 - 月K線圖暨股價漲跌資料表 - Goodinfo!台灣股市資訊網  Goodinfo!台灣股市資訊網股票代號/名稱    選擇登入方式   googletag.c)
getting history data of 2897 (492/916) with 163.172.223.200 United Kingdom  <----- ERROR (  (2897) 王道銀行 技術分析 - 月K線圖暨股價漲跌資料表 - Goodinfo!台灣股市資訊網  Goodinfo!台灣股市資訊網股票代號/名稱    選擇登入方式   googletag.)
getting history data of 4560 (665/916) with 163.172.223.200 United Kingdom  <----- ERROR (  (4560) 強信-KY 技術分析 - 月K線圖暨股價漲跌資料表 - Goodinfo!台灣股市資訊網  Goodinfo!台灣股市資訊網股票代號/名稱    選擇登入方式   googletag)
getting history data of 4807 (673/916) with 163.172.223.200 United Kingdom  <--

In [8]:
history_data_df.to_csv('history_data_20180312.csv', index=False, encoding='utf8')

In [ ]:
low_cost = history_data_df[(history_data_df.lowest > 15) & (history_data_df.lowest < 100)]

In [ ]:
low_cost.sort_values('total_money_amount', ascending=False, inplace=True)

In [ ]:
low_cost.set_index('stock_id', inplace=True)

In [ ]:
low_cost = low_cost[low_cost.total_money_amount > low_cost.total_money_amount.mean()]

In [ ]:
target_keys = list(low_cost.index)

In [ ]:
def get_latest_trade_price(code, using_proxy=False):
    try:
        timestemp = str(math.floor(time.time()*1000))

        if using_proxy:
            r = requests.get('http://mis.twse.com.tw/stock/fibest.jsp?stock=' + code, headers=headers, proxies=proxies)
            r = requests.get('http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_' + code + '.tw&json=1&delay=0&_=' + timestemp, 
                             headers=headers, cookies={'JSESSIONID': r.cookies['JSESSIONID']}, proxies=proxies)
        else:
            r = requests.get('http://mis.twse.com.tw/stock/fibest.jsp?stock=' + code, headers=headers)
            r = requests.get('http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_' + code + '.tw&json=1&delay=0&_=' + timestemp, 
                             headers=headers, cookies={'JSESSIONID': r.cookies['JSESSIONID']})

        return float(r.json()['msgArray'][0]['z'])
    except:
        soup = BeautifulSoup(r.content, 'lxml')
        print(soup.text.replace('\n', '')[:100])

In [ ]:
ress = {}
for c in target_keys:
    for i in range(try_time): # trying at most 5 times
        res = get_latest_trade_price(c, using_proxy=True) # using_proxy才會改用tor IP
        if res:
            print(c, res)
            ress[c] = res
            break
        elif i == try_time - 1:
            print('%s Failed twice, skipping.' % (c))
        else:
            print('%s Failed, getting new ip: %s' % (c, get_new_ip()))
            get_new_ip()

In [ ]:
low_cost['latest_price'] = ress.values()

In [ ]:
low_cost[low_cost.lowest > low_cost.latest_price]

# FOR TESTING

In [ ]:
get_new_ip()
r = requests.get('http://orange.tw/', proxies=proxies)
print(r.text)